In [1]:
# Загрузить библиотеки
import numpy as np
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers.convolutional import Conv2D, MaxPooling2D
from keras.utils import np_utils
from keras import backend as K

import tensorflow as tf
import datetime
import mlflow

Using TensorFlow backend.


In [2]:
# Сделать значение цветового канала первым
#K.set_image_data_format("channels_first")

# Задать информацию об изображении
channels = 1
height = 28
width = 28

In [3]:
# Загрузить данные и цель из набора данных MNIST рукописных цифр  
(data_train, target_train), (data_test, target_test) = mnist.load_data()

# # Реформировать тренировочные данные об изображениях в признаки
data_train = data_train.reshape(data_train.shape[0], height, width, channels)

# # Реформировать тестовые данные об изображениях в признаки
data_test = data_test.reshape(data_test.shape[0], height, width, channels)

# Прошкалировать пиксельную интенсивность в диапазон между 0 и 1
features_train = data_train / 255
features_test = data_test / 255

# Преобразовать цель в кодировку с одним активным состоянием
target_train = np_utils.to_categorical(target_train)
target_test = np_utils.to_categorical(target_test)
number_of_classes = target_test.shape[1]

In [4]:
# Инициализировать нейронную сеть
network = Sequential()

# Добавить сверточный слой с 64 фильтрами, окном 5x5 и 
# активационной функций ReLU
network.add(Conv2D(filters=64,
                   kernel_size=(5, 5),
                   input_shape=(width, height, channels),
                   activation='relu'))

# Добавить максимально редуцирующий слой с окном 2x2
network.add(MaxPooling2D(pool_size=(2, 2)))

# Добавить отсеивающий слой
network.add(Dropout(0.3))

# Добавить слой для сглаживания входа
network.add(Flatten())

# Добавить полносвязный слой из 128 блоков с
# активационной функций ReLU
network.add(Dense(128, activation="relu"))

# Добавить отсеивающий слой
network.add(Dropout(0.5))

# Добавить полносвязный слой c
# активационной функцией softmax
network.add(Dense(number_of_classes, activation="softmax"))

In [5]:
from logger_metric.loggers import csvlog
from logger_metric.loggers import tblog
from logger_metric.loggers import mlflog

from logger_metric.loggers import tfcallback

In [6]:
# CSV логгирование
#logger = csvlog.CSVLogger(file_path='Loss_tabl', metric_key_list=['Loss', 'Accuracy'])

# Tensorboard логгирование
#logger = tblog.TBLogger(dir_path='TB_metric', metric_key_list =['Loss', 'Accuracy'])

# MLflow логгирование
logger = mlflog.MLFLogger(dir_path='d:/7777/MLF_metric', metric_key_list =['Loss', 'Accuracy'])


logger_callback = tfcallback.LoggerCallback(logger=logger, field_name_mapping={'Loss': 'loss', 'Accuracy': 'accuracy'})

In [7]:
 # Скомплировать нейронную сеть
network.compile(
    loss="categorical_crossentropy", # Перекрестная энтропия
    optimizer="rmsprop",  # Распространение СКО
    metrics=["accuracy"]) # Точностный показатель результативности

In [9]:
with mlflow.start_run() as run:

    # Натренировать нейронную сеть
    history = network.fit(
        features_train,  # Признаки
        target_train,    # Цель
        epochs=7,        # Количество эпох
        verbose=1,       # Печатать описание после каждой эпохи
        batch_size=1000, # Количество наблюдений на пакет
        validation_data=(features_test, target_test), # Данные для оценивания
        callbacks=[logger_callback]) 

2021/08/23 23:19:19 WARNING mlflow.tracking.context.git_context: Failed to import Git (the Git executable is probably not on your PATH), so Git SHA is not available. Error: Failed to initialize: Bad git executable.
The git executable must be specified in one of the following ways:
    - be included in your $PATH
    - be set via $GIT_PYTHON_GIT_EXECUTABLE
    - explicitly set via git.refresh()

All git commands will error until this is rectified.

This initial warning can be silenced or aggravated in the future by setting the
$GIT_PYTHON_REFRESH environment variable. Use one of the following values:
    - quiet|q|silence|s|none|n|0: for no warning or exception
    - warn|w|warning|1: for a printed warning
    - error|e|raise|r|2: for a raised exception

Example:
    export GIT_PYTHON_REFRESH=quiet



Epoch 1/7
60/60 [==============================] - 5s 54ms/step - loss: 0.9394 - accuracy: 0.6982 - val_loss: 0.1460 - val_accuracy: 0.9583
Epoch 2/7
60/60 [==============================] - 2s 41ms/step - loss: 0.1928 - accuracy: 0.9447 - val_loss: 0.0739 - val_accuracy: 0.9763
Epoch 3/7
60/60 [==============================] - 2s 41ms/step - loss: 0.1141 - accuracy: 0.9661 - val_loss: 0.0566 - val_accuracy: 0.9824
Epoch 4/7
60/60 [==============================] - 2s 41ms/step - loss: 0.0913 - accuracy: 0.9732 - val_loss: 0.0510 - val_accuracy: 0.9820
Epoch 5/7
60/60 [==============================] - 2s 41ms/step - loss: 0.0703 - accuracy: 0.9789 - val_loss: 0.0407 - val_accuracy: 0.9866
Epoch 6/7
60/60 [==============================] - 2s 41ms/step - loss: 0.0631 - accuracy: 0.9808 - val_loss: 0.0368 - val_accuracy: 0.9875
Epoch 7/7
60/60 [==============================] - 2s 41ms/step - loss: 0.0518 - accuracy: 0.9848 - val_loss: 0.0340 - val_accuracy: 0.9887


In [10]:
%tensorboard --logdir='TB_metric'

UsageError: Line magic function `%tensorboard` not found.
